# Receitas
## Bloco de Questões I - SQL Básico

Considere os comandos SQL a seguir para criar tabelas que controlam Produtos e Receitas, bem como o respectivo esquema relacional simplificado abaixo. A tabela de Produto mantém um cadastro de produtos, com seu código, nome e custo_unitario, que corresponde ao custo de aquisição de uma unidade do produto. Cada Receita tem um código, nome, tipo (e.g., vegana, regular, light) e custo_total (corresponde ao custo de produção da receita inteira). Cada entrada na tabela Ingrediente indica que um Produto é componente de uma Receita em uma certa quantidade.

### Esquema Relacional
~~~
Produto(codigo_produto, nome_produto, custo_unitario)
Receita(codigo_receita, nome_receita, tipo_receita, custo_total)
Ingrediente(codigo_receita, codigo_produto, quantidade)
~~~

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP Table IF EXISTS Ingrediente;
DROP Table IF EXISTS Receita;
DROP Table IF EXISTS Produto;

CREATE TABLE Produto (
  codigo_produto VARCHAR(7),
  nome_produto VARCHAR(80),
  custo_unitario DECIMAL(5,2),
  PRIMARY KEY (codigo_produto)
);

CREATE TABLE Receita (
  codigo_receita VARCHAR(7),
  nome_receita VARCHAR(80),
  tipo_receita VARCHAR(10),
  custo_total DECIMAL(5,2),
  PRIMARY KEY (codigo_receita));
               
CREATE TABLE Ingrediente (
  codigo_receita VARCHAR(7),
  codigo_produto VARCHAR(7),
  quantidade DECIMAL(5,2),
  PRIMARY KEY (codigo_receita, codigo_produto, quantidade),
  FOREIGN KEY (codigo_receita)
    REFERENCES Receita (codigo_receita),
  FOREIGN KEY (codigo_produto)
    REFERENCES Produto (codigo_produto) );

INSERT INTO Produto VALUES ('CHOCO', 'Chocolate', 3.0);
INSERT INTO Produto VALUES ('CENO', 'Cenoura', 1.5);
INSERT INTO Produto VALUES ('ACU', 'Acucar', 0.5);
INSERT INTO Produto VALUES ('SUCRA', 'Sucralose', 5.0);
INSERT INTO Produto VALUES ('FAR', 'Farinha', 1.0);

INSERT INTO Receita VALUES ('BOLOCE', 'Bolo Cenoura', 'vegana', 6.0);
INSERT INTO Receita VALUES ('BOLOCH', 'Bolo Chocolate', 'regular', 6.7);
INSERT INTO Receita VALUES ('BOLOCEL', 'Bolo Cenoura Light', 'light', 10.0);

INSERT INTO Ingrediente VALUES ('BOLOCE', 'CENO', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCE', 'ACU', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCE', 'FAR', 3.0);
INSERT INTO Ingrediente VALUES ('BOLOCE', 'ACU', 2.0);
INSERT INTO Ingrediente VALUES ('BOLOCH', 'CHOCO', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCH', 'ACU', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCH', 'FAR', 3.0);
INSERT INTO Ingrediente VALUES ('BOLOCEL', 'CENO', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCEL', 'SUCRA', 1.0);
INSERT INTO Ingrediente VALUES ('BOLOCEL', 'FAR', 3.0);

SELECT * FROM Produto;
SELECT * FROM Receita;
SELECT * FROM Ingrediente;

## Questão 1

Liste o código de todas as receitas e o código dos ingredientes das receitas.

In [3]:
SELECT I.CODIGO_RECEITA, I.CODIGO_PRODUTO
FROM Ingrediente I;

## Questão 2

Liste o nome de todas as receitas e o nome dos ingredientes das receitas.

In [4]:
SELECT R.NOME_RECEITA, P.NOME_PRODUTO
FROM Receita R, Ingrediente I, Produto P
WHERE R.CODIGO_RECEITA = I.CODIGO_RECEITA AND I.CODIGO_PRODUTO = P.CODIGO_PRODUTO;

## Questão 3
Liste o nome de todos os produtos que aparecem em receitas veganas, cujo custo unitário é maior que 1.0. Nomes de produtos não devem aparecer duplicados.

In [5]:
SELECT DISTINCT P.nome_produto
FROM Produto P, Ingrediente I, Receita R
WHERE R.codigo_receita=I.codigo_receita AND P.codigo_produto=I.codigo_produto
      AND R.tipo_receita='vegana' AND P.custo_unitario > 1.0

Cenoura

# Questã 4

Liste o nome de todas as receitas e o número de ingredientes que ela possui.

In [6]:
SELECT R.nome_receita, COUNT(I.codigo_produto)
FROM Receita R, Ingrediente I
WHERE R.codigo_receita = I.codigo_receita
GROUP BY R.codigo_receita;

## Questão 5

Em algumas receitas o mesmo produto aparece mais de uma vez com quantidades diferentes. Apresente a mesma tabela de Ingredientes de modo que não haja produtos que aparecem mais de uma vez. Para isso, junte os produtos que aparecem mais de uma vez na mesma receita e some as suas quantidades.

In [7]:
SELECT CODIGO_RECEITA, CODIGO_PRODUTO, SUM(QUANTIDADE) QUANTIDADE
FROM INGREDIENTE
GROUP BY CODIGO_RECEITA, CODIGO_PRODUTO;

## Questão 6
Liste o nome dos Produtos que aparecem em mais de uma Receita.

In [8]:
SELECT P.nome_produto
FROM PRODUTO P, INGREDIENTE I
WHERE P.CODIGO_PRODUTO = I.CODIGO_PRODUTO
GROUP BY I.CODIGO_PRODUTO
HAVING COUNT(*) >= 2;

## Questão 7

A coluna `custo_total` de uma Receita deveria calculado pelo somatório do `custo_unitario` de cada ingrediente multiplicado pela sua quantidade na receita. Escreva para verificar se a coluna está correta. A consulta apresenta o nome de todas as receitas cujo custo_total não atende a este critério, seu custo presente na `coluna_total` e seu custo calculado a patir dos ingredientes.

Tente fazer a questão diretamente. Se não conseguir, resolva o passo a passo a seguir (6.1 a 6.3).

In [9]:
SELECT R.NOME_RECEITA, R.CUSTO_TOTAL, SUM(I.quantidade * P.custo_unitario) CUSTO_CALCULADO
FROM Receita R, Ingrediente I, Produto P
WHERE R.CODIGO_RECEITA = I.CODIGO_RECEITA AND I.CODIGO_PRODUTO = P.CODIGO_PRODUTO
GROUP BY I.CODIGO_RECEITA
HAVING R.CUSTO_TOTAL <> CUSTO_CALCULADO;

### Questão 7 - Passo 1
Liste o nome de todas as receitas e o seu custo total (coluna `custo_total`).

In [10]:
SELECT R.NOME_RECEITA, R.CUSTO_TOTAL
FROM RECEITA R;

### Questão 7 - Passo 2
Amplie a query do passo 1 acrescentando uma terceira coluna de custo calculado a partir dos ingredientes. Esse custo é calculado pelo somatório do custo de cada ingrediente multiplicado pela sua quantidade na receita.

In [11]:
SELECT R.NOME_RECEITA, R.CUSTO_TOTAL, SUM(I.quantidade * P.custo_unitario) CUSTO_CALCULADO
FROM Receita R, Ingrediente I, Produto P
WHERE R.CODIGO_RECEITA = I.CODIGO_RECEITA AND I.CODIGO_PRODUTO = P.CODIGO_PRODUTO
GROUP BY I.CODIGO_RECEITA;

### Questão 7 - Passo 3

Apresente aquelas receitas em que o valor da coluna `custo_total` não corresponde ao custo calculado.

In [12]:
SELECT R.NOME_RECEITA, R.CUSTO_TOTAL, SUM(I.quantidade * P.custo_unitario) CUSTO_CALCULADO
FROM Receita R, Ingrediente I, Produto P
WHERE R.CODIGO_RECEITA = I.CODIGO_RECEITA AND I.CODIGO_PRODUTO = P.CODIGO_PRODUTO
GROUP BY I.CODIGO_RECEITA
HAVING R.CUSTO_TOTAL <> CUSTO_CALCULADO;

# Bloco de Questões sobre JOIN Explícito

## Questão 2b

Considerando a questão 2: liste o nome de todas as receitas e o nome dos ingredientes das receitas. Modifique-a de modo a apresentá-la com JOIN Explícito.

In [13]:
SELECT R.NOME_RECEITA, P.NOME_PRODUTO
FROM Receita R JOIN Ingrediente I ON R.CODIGO_RECEITA = I.CODIGO_RECEITA
               JOIN Produto P ON I.CODIGO_PRODUTO = P.CODIGO_PRODUTO;